1\. **Maximum wind speed prediction at the Sprogø station**

The exercise goal is to predict the maximum wind speed occurring every 50 years even if no measure exists for such a period. The available data are only measured over 21 years at the Sprogø meteorological station located in Denmark. 

The annual maxima are supposed to fit a normal probability density function. However such function is not going to be estimated because it gives a probability from a wind speed maxima. Finding the maximum wind speed occurring every 50 years requires the opposite approach, the result needs to be found from a defined probability. That is the quantile function role and the exercise goal will be to find it. In the current model, it is supposed that the maximum wind speed occurring every 50 years is defined as the upper 2% quantile.

By definition, the quantile function is the inverse of the cumulative distribution function. The latter describes the probability distribution of an annual maxima. In the exercise, the cumulative probability $p_i$ for a given year i is defined as $p_i = i/(N+1)$ with $N = 21$, the number of measured years. Thus it will be possible to calculate the cumulative probability of every measured wind speed maxima. From those experimental points, the scipy.interpolate module will be very useful for fitting the quantile function. Finally the 50 years maxima is going to be evaluated from the cumulative probability of the 2% quantile.

Practically, load the dataset:

```python
import numpy as np
max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
```

Compute then the cumulative probability $p_i$ (`cprob`) and sort the maximum speeds from the data. Use then the  UnivariateSpline from scipy.interpolate to define a quantile function and thus estimate the probabilities.

In the current model, the maximum wind speed occurring every 50 years is defined as the upper 2% quantile. As a result, the cumulative probability value will be:

```python
fifty_prob = 1. - 0.02
```

So the storm wind speed occurring every 50 years can be guessed as:

``` python
fifty_wind = quantile_func(fifty_prob)
```



In [ ]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from scipy.interpolate import UnivariateSpline

##### Exercise 1)

In [ ]:
max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
lin = np.linspace(0,22,21)
cprob = [i/(years_nb+1) for i in lin]
ordered_speeds = np.sort(max_speeds)
quantile = UnivariateSpline (cprob,ordered_speeds)


plt.plot(quantile(cprob),cprob)
plt.xlabel('Wind Speed')
plt.ylabel('Probabily Distribution')
plt.grid()

fifty_prob = 1. - 0.02
fifty_wind = quantile(fifty_prob)
print('Storm wind speed occurring every 50 years = ',fifty_wind,'km/h')


2\. **Curve fitting of temperature in Alaska** 

The temperature extremes in Alaska for each month, starting in January, are given by (in degrees Celcius):

max:  17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18

min: -62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58

* Plot these temperature extremes.
* Define a function that can describe min and max temperatures. 
* Fit this function to the data with scipy.optimize.curve_fit().
* Plot the result. Is the fit reasonable? If not, why?
* Is the time offset for min and max temperatures the same within the fit accuracy?

##### Exercise 2)

In [ ]:
from matplotlib.dates import MonthLocator

### Display Data
temp_min = np.array([17, 19, 21, 28, 33, 38, 37, 37, 31, 23, 19, 18])
temp_max = np.array([-62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58])

plt.figure(figsize=(6, 4))
lin = np.linspace(1,12,12)
plt.plot(lin,temp_min,'*',lin,temp_max,'o')

### interpolation
from scipy.interpolate import interp1d

# linear interpolation
linear_interp_min = interp1d(lin, temp_min)
linear_interp_max = interp1d(lin, temp_max)
# cubic interpolation
cubic_interp_min = interp1d(lin, temp_min, kind='cubic')
cubic_interp_max = interp1d(lin, temp_max, kind='cubic')
# Plot result
# Linear
linear_interp_min_result = linear_interp_min(lin)
linear_interp_max_result = linear_interp_max(lin)
plt.plot(lin, linear_interp_min_result, label='linear interp min')
plt.plot(lin, linear_interp_max_result, label='linear interp max')
# Cubic
#cubic_interp_min_result = cubic_interp_min(lin)
#cubic_interp_max_result = cubic_interp_max(lin)
#plt.plot(lin, cubic_interp_min_result, label='cubic interp min')
#plt.plot(lin, cubic_interp_max_result, label='cubic interp max')

plt.xlabel('Month')
plt.ylabel('Temperture(C)')
plt.title('Linear Interpolation')
plt.grid()
plt.legend()
plt.show()

### Define  (Bell curve)
import matplotlib.mlab as mlab
plt.figure(figsize=(6, 4))

mu_min = 7
mu_max = 7
variance_min = 2.5
variance_max = 3
sigma_min = np.sqrt(variance_min)
sigma_max = np.sqrt(variance_max)

plt.plot(lin,temp_min,'*',lin,temp_max,'o')
plt.plot(lin,200*(mlab.normpdf(lin, mu_min, sigma_min))-60,label = 'Fitting function min')
plt.plot(lin,100*(mlab.normpdf(lin, mu_max, sigma_max))+15,label = 'Fitting function max')

plt.xlabel('Month')
plt.ylabel('Temperture(C)')
plt.title('Guess Function')
plt.grid()
plt.legend()
plt.show()

### Optimize fitting
def f(lin,A,variance,b):
    mu = 7
    return A*(mlab.normpdf(lin, mu, np.sqrt(abs(variance))))+b

from scipy import optimize
params_min, params_covariance_min = optimize.curve_fit(f,lin,temp_min)
params_max, params_covariance_max = optimize.curve_fit(f,lin,temp_max)
print("Amplitude,Variance,Bias = (Min temp)",params_min)
print("Amplitude,Variance,Bias = (Max Temp)",params_max)

plt.figure(figsize=(6, 4))
plt.plot(lin,temp_min,'*',lin,temp_max,'o')
plt.plot(lin, f(lin,params_min[0], params_min[1],params_min[2]),label='Fitted function max')
plt.plot(lin, f(lin,params_max[0], params_max[1],params_max[2]),label='Fitted function min')
plt.xlabel('Month')
plt.ylabel('Temperture(C)')
plt.title('Optimize Fitting Function')
plt.grid()
plt.legend()
plt.show()

3\. **2D minimization of a six-hump camelback function**

$$
f(x,y) = \left(4-2.1x^2+\frac{x^4}{3} \right) x^2 +xy + (4y^2 -4)y^2
$$

has multiple global and local minima. Find the global minima of this function.

Hints:

* Variables can be restricted to $-2 < x < 2$ and $-1 < y < 1$.
* Use numpy.meshgrid() and pylab.imshow() to find visually the regions.
* Use scipy.optimize.minimize(), optionally trying out several of its methods.

How many global minima are there, and what is the function value at those points? What happens for an initial guess of $(x, y) = (0, 0)$ ?


##### Answer 3)

In [ ]:
import pylab
def f(x,y):
    return ((4-2.1*x**2+x**4/3))*(x**2)+x*y+(4*y**2-4)*y**2
x = np.linspace(-2,2,50)
y = np.linspace(-1,1,50)

### Plot Regarding Meshgrid Description
xv,yv = np.meshgrid(x,y,sparse=True)
z = f(xv,yv)
h = plt.contourf(x,y,z)
plt.show()

# Bounded Solution:
a = optimize.minimize_scalar(f, method='bounded', args=(0.75,),bounds=[-2, 2])
print("\nBounded results for (y= 0.75)\n",a.x)
b = optimize.minimize_scalar(f, method='bounded', args=(-0.75,),bounds=[-2, 2])
print("\nBounded results for (y= -0.75)\n",b.x)

# Using Brackets without oscilation
solutions = optimize.minimize_scalar(f, args=(0,),bracket=(-1,0,1))
print("\nBounded results without oscilation\n",solutions)

# Using Brackets with oscilation
lower = np.random.uniform(-2, 2, 100)
upper = lower + 0.1
solutions = [optimize.minimize_scalar(f, args=(0,), bracket=(l, u)) for (l, u) in zip(lower, upper)]
idx = np.argmin([solution.fun for solution in solutions])
solution = solutions[idx]
print("\nBounded results with oscilation\n",solution)

# Brent method
c = [optimize.minimize_scalar(f,method='Brent',args=(l,)) for l in range(-1,1,1000)]
print("\nBrent method results\n",c)

4\. **FFT of a simple dataset**

Performe a periodicity analysis on the lynxs-hares population

##### Asnwer 4)

In [ ]:
#Load the data 
population = np.loadtxt('populations.txt')
years = population[:,0]
hares = population[:,1]
lynxes = population[:,2]
carrots = population[:,3]

#Plot the 3 populations over the years
plt.plot(years,hares,years,lynxes,years,carrots)
labels = ['hares', 'lynxes', 'carrots']
plt.xlabel('Years')
plt.ylabel('Measurment')
year = np.linspace(np.min(years),np.max(years),5)
plt.xticks(year)
plt.legend(labels)
plt.grid()
plt.title('Population')
plt.show()

### FFT
from scipy import fftpack
# The FFT of the signal
fft_hares = fftpack.fft(hares)
fft_lyn = fftpack.fft(lynxes)
fft_car = fftpack.fft(carrots)

# And the power (fft is of complex dtype)
power_hares = np.abs(fft_hares)
power_lyn = np.abs(fft_lyn)
power_car = np.abs(fft_car)

# The corresponding frequencies
sample_freq_hares = fftpack.fftfreq(hares.size, d=years)
sample_freq_lyn = fftpack.fftfreq(lynxes.size, d=years)
sample_freq_car = fftpack.fftfreq(carrots.size, d=years)

# Plot the FFT power
plt.figure(figsize=(6, 5))
plt.plot(sample_freq_hares, power_hares,'-o')
plt.plot(sample_freq_car,power_lyn,'-^')
plt.plot(sample_freq_lyn,power_car,'--*')
plt.xlabel('Frequency [Hz]')
plt.ylabel('power')


5\. **FFT of an image**

* Examine the provided image `moonlanding.png`, which is heavily contaminated with periodic noise. In this exercise, we aim to clean up the noise using the Fast Fourier Transform.
* Load the image using pylab.imread().
* Find and use the 2-D FFT function in scipy.fftpack, and plot the spectrum (Fourier transform of) the image. Do you have any trouble visualising the spectrum? If so, why?
* The spectrum consists of high and low frequency components. The noise is contained in the high-frequency part of the spectrum, so set some of those components to zero (use array slicing).
* Apply the inverse Fourier transform to see the resulting image.

##### Answer 5)

In [ ]:
moonlanding = pylab.imread('moonlanding.png')
plt.imshow(moonlanding)
plt.show()

### Compute FFT
fft_pic = fftpack.fft2(moonlanding).real
### Compute the power and freq
power = np.abs(fftpack.fft2(moonlanding))
### Plot Spectrum and Image show
plt.imshow(fft_pic)
plt.xlabel('Image Width')
plt.ylabel('Image Length')
plt.show()

plt.plot(power)
plt.xlabel('Frequency [Hz]')
plt.ylabel('plower')
plt.title('Spectrum')
plt.show()

### Filter Image
# Make a copy not to lose the data
fft2_pic = fft_pic.copy()
r, c = fft2_pic.shape
fraction = 0.2
fft2_pic[int(r*fraction):int(r*(1-fraction)),:] = 0
fft2_pic[:, int(c*fraction):int(c*(1-fraction))] = 0

# Reconstruct the image
new_pic = fftpack.ifft2(fft2_pic).real

plt.imshow(new_pic)
plt.xlabel('Image Width')
plt.ylabel('Image Length')
plt.show()
